# Tutorial Neural Horizon MPC with acados

## Imports

In [1]:
import numpy as np
import os
import torch

from bokeh.io import output_notebook, show
output_notebook()


from src.parameters import AMPC_Param, NH_AMPC_Param, get_dataset_name
from src.mpc_classes_acados import NH_AMPC_class, get_AMPC_trajectory 
from src.inverted_pendulum_acados import acados_model_RK4, nonlinear_pend_dynamics_acados

from src.data_generation_acados import AMPC_dataset_gen
from src.neural_horizon import NN_for_casadi, load_NN
from src.mpc_dataclass import AMPC_data

from src.utils import get_features_and_labels, save_results
from src.torch_utils import count_parameters

from src.plotting import plot_MPC_results, get_figure_size

Loading BokehJS ...

## Paths

In [2]:
RESULTS_DIR = os.path.abspath('Test_Results')
DATASETS_DIR = os.path.join(RESULTS_DIR, 'MPC_data_gen')
NNS_DIR = os.path.join(RESULTS_DIR, 'Trained_Networks')
PRUNED_NNS_DIR = os.path.join(RESULTS_DIR, 'Pruned_Networks')
MPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'NH_AMPC_results')

## General setup

In [5]:
MPC_PARAM_DICT = {
    'T_sim': 5, # length of the closed-loop simulation (in seconds)
}

# Dataset Stuff
dataset_samples = 30_000

DATASET_VERSIONS = (1,2)                    # versions for training and testing e.g. 
dataset_horizon = 70                   # horizon of dataset

dataset_name = 'RTI_PCHPIPM_DISCRETE'
dataset_solver_options = dict(
    qp_solver='PARTIAL_CONDENSING_HPIPM', 
    integrator_type='DISCRETE', 
    nlp_solver_type='SQP_RTI',
    use_iter_rti_impl=True,
    use_initial_guesses=True,
    rti_tol=1e-6
)

# Neural Horizon MPC Parameters
nh_ampc_params = NH_AMPC_Param(
    # Param
    N_MPC = 8, 
    # N_NN = 17 -> N = N_NN + M = 17 + 8
    N = 25,                                     

    # Dataset stuff
    N_DS = dataset_horizon, 
    TRAIN_V_DS = DATASET_VERSIONS[0], 
    TEST_V_DS = DATASET_VERSIONS[1], 
    DS_samples = dataset_samples,
    DS_opts_name = dataset_name,
    DS_begin = 'fixed',
    DS_feature = 8,

    # NN stuff
    N_hidden = 48,

    # Param
    **MPC_PARAM_DICT
)

# Acados Options
solver_name = 'ASRTID_FCH'
solver_opotions = dict(
    qp_solver='FULL_CONDENSING_HPIPM', 
    integrator_type='DISCRETE', 
    nlp_solver_type='SQP_RTI',
    as_rti_iter=3,
    as_rti_level=3,
    nlp_solver_tol_stat=1e-6,
    nlp_solver_max_iter=3,
)

# Trajectory results path
results_filename = f'NH_AMPC_results_{solver_name}_{nh_ampc_params.NN_name}'
nh_ampc_results_path = os.path.join(MPC_RESULTS_DIR, results_filename)

### Cuda use for training

In [ ]:
USE_CUDA = True
device = torch.device('cuda:0' if torch.cuda.is_available() and USE_CUDA else 'cpu') 
dtype = torch.float32
print(device)

## Dataset generation

In [ ]:
for dataset_version in DATASET_VERSIONS:
    
    print('#' + '=' * 80)
    print(f'{dataset_samples} training data samples for a horizon of {dataset_horizon} -> Version {dataset_version}')
    print(f'Acados options: {dataset_name}')

    ampc_param = AMPC_Param(N_MPC=dataset_horizon, **MPC_PARAM_DICT)
    model = acados_model_RK4(nonlinear_pend_dynamics_acados(ampc_param), ampc_param)
    sampler = AMPC_dataset_gen(
        model,
        ampc_param, 
        solver_options=dataset_solver_options,
        n_samples=dataset_samples,
        chance_reset_sample=0.25,
        init_scale=np.array([.75, .15, .25, .25])
    )
    gen_file = sampler.generate_data(
        filename=get_dataset_name(dataset_horizon, dataset_samples, dataset_name, dataset_version),
        filedir=DATASETS_DIR
    )
    sampler.MPC.cleanup()
    del sampler.MPC

## Neural Network generation

### Train Network

In [ ]:
train_dataset_file = os.path.join(DATASETS_DIR, nh_ampc_params.train_DS_name)

features, labels = get_features_and_labels(nh_ampc_params)
NN_fc = NN_for_casadi(
    train_dataset_file, 
    nh_ampc_params, 
    features=features,
    labels=labels,
    device=device, 
    dtype=dtype
)
NN_fc.NNcompile(show_tqdm=True, n_neurons=nh_ampc_params.N_hidden)
NN_fc.NNsave(file=nh_ampc_params.NN_name, filedir=NNS_DIR)

### Evaluate Network

In [ ]:
trained_NN_file = os.path.join(NNS_DIR, nh_ampc_params.NN_name)
NN_fc = NN_for_casadi.NNload(trained_NN_file, P=nh_ampc_params, device=device, dtype=dtype)

test_datasets_file = os.path.join(DATASETS_DIR, nh_ampc_params.test_DS_name)
r2_score, relative_error = NN_fc.evaluate_NN(test_datasets_file)

print(
f"""
    N_NN             = {nh_ampc_params.N_NN}
    N_hidden         = {nh_ampc_params.N_hidden}
    Version          = {nh_ampc_params.V_NN},
    R2_score         = {r2_score:.4f}, 
    Rel_err_mean     = {100*relative_error.mean():.2f},
    Rel_err_std      = {100*relative_error.std():.2f},
    NN_param_size    = {count_parameters(NN_fc.NN)},
"""
)

## Neural Horizon MPC with acados trajectory generation

In [ ]:
NN_fc = load_NN(nh_ampc_params, NNS_DIR, DATASETS_DIR, device, dtype)
NN_fc.evaluate_NN(os.path.join(DATASETS_DIR, nh_ampc_params.test_DS_name))

model = acados_model_RK4(nonlinear_pend_dynamics_acados(nh_ampc_params), nh_ampc_params)
cAMPC = NH_AMPC_class(
    model,
    NN_fc, 
    solver_options=solver_opotions, 
    horizon_name=nh_ampc_params.param_name, 
    acados_name=solver_name
)
try:
    results = get_AMPC_trajectory(cAMPC, show_tqdm=False, verbose=False)
except Exception as e:
    raise e
finally:
    cAMPC.cleanup()
    del cAMPC

save_results(nh_ampc_results_path, results, always_overwrite=True)

## Show results

In [6]:
results = AMPC_data.load(nh_ampc_results_path)

use_latex_style = False
figure_size = get_figure_size(fraction=1.0, ratio=5.) if use_latex_style else (1200, 200)

p_res = plot_MPC_results(
    [results],
    plot_mpc_trajectories=True, 
    xbnd=1.5, 
    thickness=[3 for _ in range(3)],
    dash=['solid' for _ in range(3)],
    # solver_time_scale='linear',
    additional_plots=['Iterations', 'Prep_Time', 'Fb_Time'],
    additional_plots_options={
        'Iterations': {'y_axis_label': r'$$i_{\mathrm{sol}}$$'}, 
        'Prep_Time': {'y_axis_label': r'$$t_{\mathrm{prep}} \, [s]$$'},
        'Fb_Time': {'y_axis_label': r'$$t_{\mathrm{fb}} \, [s]$$'}, 
    },
    width=figure_size[0],
    height=figure_size[1],
    latex_style=use_latex_style
)
show(p_res)